In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import datetime

In [30]:
START_POI = 0
END_POI = 400
TOTAL_DAYS = 400
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 1-(TRAIN_RATIO+VAL_RATIO)
START_DATE = '2018-12-31'
WINDOW_SIZE = 24
HORIZON = 6
DATASET = 'Houston'

In [31]:
def masked_MAPE(v, v_, axis=None):
    '''
    Mean absolute percentage error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, MAPE averages on all elements of input.
    '''
    # TODO: Remove this line
    v_ = np.where(v_<0, 0,v_)
    
    mask = (v < 1E-5)
    percentage = np.abs(v_ - v) / np.abs(v)
    percentage = np.where(percentage > 5, 5, percentage) # TODO remove this
    if np.any(mask):
        masked_array = np.ma.masked_array(percentage, mask=mask)  # mask the dividing-zero as invalid
        result = masked_array.mean(axis=axis)
        if isinstance(result, np.ma.MaskedArray):
            return result.filled(np.nan)
        else:
            return result
    return np.mean(percentage, axis).astype(np.float64)


def MAPE(v, v_, axis=None):
    '''
    Mean absolute percentage error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, MAPE averages on all elements of input.
    '''
    # TODO: Remove this line
    v_ = np.where(v_<0, 0,v_)
    mape = (np.abs(v_ - v) / np.abs(v)+1e-5).astype(np.float64)
    mape = np.where(mape > 5, 5, mape)
    return np.mean(mape, axis)


def RMSE(v, v_, axis=None):
    '''
    Mean squared error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, RMSE averages on all elements of input.
    '''
    return np.sqrt(np.mean((v_ - v) ** 2, axis)).astype(np.float64)




def MAE(v, v_, axis=None):
    '''
    Mean absolute error.
    :param v: np.ndarray or int, ground truth.
    :param v_: np.ndarray or int, prediction.
    :param axis: axis to do calculation.
    :return: int, MAE averages on all elements of input.
    '''

    return np.mean(np.abs(v_ - v), axis).astype(np.float64)


def evaluate(y, y_hat, by_step=False, by_node=False):
    '''
    :param y: array in shape of [count, time_step, node].
    :param y_hat: in same shape with y.
    :param by_step: evaluate by time_step dim.
    :param by_node: evaluate by node dim.
    :return: array of mape, mae and rmse.
    '''
    if not by_step and not by_node:
        return masked_MAPE(y, y_hat), MAE(y, y_hat), RMSE(y, y_hat)
    if by_step and by_node:
        return masked_MAPE(y, y_hat, axis=0), MAE(y, y_hat, axis=0), RMSE(y, y_hat, axis=0)
    if by_step:
        return masked_MAPE(y, y_hat, axis=(0, 2)), MAE(y, y_hat, axis=(0, 2)), RMSE(y, y_hat, axis=(0, 2))
    if by_node:
        return masked_MAPE(y, y_hat, axis=(0, 1)), MAE(y, y_hat, axis=(0, 1)), RMSE(y, y_hat, axis=(0, 1))

In [32]:
val_prediction_path = f'../../output/baselines/{DATASET}/historical_avg_forecasts.csv'
val_target_path = f'../../output/baselines/{DATASET}/target.csv'

In [34]:
val_predict_df = pd.read_csv(val_prediction_path, header=None).to_numpy()
val_target_df = pd.read_csv(val_target_path, header=None).to_numpy()

In [40]:
val_predict_df[:, :20].shape

(1892, 20)

In [37]:
evaluate(val_target_df, val_predict_df)

/tmp/ipykernel_31749/3177913784.py:13: RuntimeWarning: divide by zero encountered in true_divide
  percentage = np.abs(v_ - v) / np.abs(v)
/tmp/ipykernel_31749/3177913784.py:13: RuntimeWarning: invalid value encountered in true_divide
  percentage = np.abs(v_ - v) / np.abs(v)


(0.7830078646799231, 8.86041754756871, 26.911243936529885)